In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog


#model loading

model = tf.keras.models.load_model('shapedetector_model.h5')

#show the model architecture 
#shapedetector_model.summary()

class_names = ['circle', 'rectangle', 'square', 'triangle']

img_height = 28
img_width = 28

#Gui 


def load_img():
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="predictiondata", title="Choose an image",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png")))
    
    basewidth = 300
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper()).pack()
    panel_image = tk.Label(frame, image=img).pack()


    
def classify():
    img = keras.preprocessing.image.load_img(
    image_data, target_size=(img_height, img_width))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    label = "This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score))
    table = tk.Label(frame, text="Top image class predictions and confidences").pack()
    result = tk.Label(frame,text= label).pack()



root = tk.Tk()
root.title('Shape Detector')

root.resizable(False, False)
tit = tk.Label(root, text="Shape Detector", padx=25, pady=6, font=("", 12)).pack()
canvas = tk.Canvas(root, height=500, width=500, bg='grey')
canvas.pack()
frame = tk.Frame(root, bg='white')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)
chose_image = tk.Button(root, text='Choose Image',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=load_img)
chose_image.pack(side=tk.LEFT)
class_image = tk.Button(root, text='Classify Image',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=classify)
class_image.pack(side=tk.RIGHT)

root.mainloop()